In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_vae_lib
import inv_kl_objective_lib as inv_kl_objective_lib

import kl_objective_lib 

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

In [ ]:
# load PSF
psf_fit_file = '../../celeste_net/sdss_stage_dir/3900/6/269/psField-003900-6-0269.fit'
print('psf file: \n', psf_fit_file)

In [ ]:
np.random.seed(43534)
_ = torch.manual_seed(24534)

# Draw data

In [ ]:
# data parameters
with open('../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)

data_params['slen'] = 101
data_params['min_stars'] = 2000
data_params['max_stars'] = 2000
data_params['alpha'] = 0.5

print(data_params)


In [ ]:
max_stars = data_params['max_stars']

In [ ]:
n_images = 4

star_dataset = \
    simulated_datasets_lib.load_dataset_from_params(psf_fit_file,
                            data_params,
                            n_images = n_images,
                            add_noise = True)

num_unlabeled = star_dataset.fluxes.shape[0]
print('num unlabeled', num_unlabeled)

In [ ]:
# get loader 
batchsize = n_images

loader = torch.utils.data.DataLoader(
                 dataset=star_dataset,
                 batch_size=batchsize,
                 shuffle=False)

loader.dataset.set_params_and_images()

In [ ]:
for _, data in enumerate(loader):
    true_full_fluxes = data['fluxes']
    true_full_locs = data['locs']
    full_images = data['image']
    full_backgrounds = torch.zeros(full_images.shape) * data_params['sky_intensity']
        
    break

In [ ]:
for i in range(1): 
    plt.matshow(full_images[i, 0, :, :])

# Define VAE

In [ ]:
star_encoder = starnet_vae_lib.StarEncoder(full_slen = data_params['slen'],
                                           stamp_slen = 9,
                                           step = 2,
                                           edge_padding = 3,
                                           n_bands = 1,
                                           max_detections = 4)


In [ ]:
star_encoder.load_state_dict(torch.load('../fits/starnet_invKL_encoder-10092019-reweighted_samples',
                               map_location=lambda storage, loc: storage))
star_encoder.eval(); 

In [ ]:
loss, counter_loss, locs_loss, fluxes_loss, perm = \
    inv_kl_objective_lib.get_encoder_loss(star_encoder,
                                            full_images,
                                            full_backgrounds,
                                            true_full_locs,
                                            true_full_fluxes, use_l2_loss = False)

In [ ]:
import inv_kl_objective_lib_deleteme

In [ ]:
loss2, counter_loss2, locs_loss2, fluxes_loss2, perm = \
    inv_kl_objective_lib_deleteme.get_encoder_loss(star_encoder,
                                            full_images,
                                            full_backgrounds,
                                            true_full_locs,
                                            true_full_fluxes, use_l2_loss = False)

In [ ]:
assert loss == loss2

In [ ]:
assert torch.all(counter_loss == counter_loss2)

In [ ]:
(locs_loss - locs_loss2).abs().max()

In [ ]:
(fluxes_loss - fluxes_loss2).abs().max()